In [ ]:
%load_ext autoreload
%autoreload 1
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy
import scipy.optimize
import threading
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
%aimport read_tags, util
from read_tags import Detector, get_local_ip
import util

import sys
sys.path.append('..')
%aimport gerry09.communicate
from gerry09.communicate import CableRobot, MotorState, ControllerState

In [ ]:
PORT = '/dev/tty.usbmodem103568503'
print(f'Connect AprilTag iOS app to {get_local_ip()}')
W_in, H_in = 227.25, 12 * 12 + 0.75 - 3
W, H = W_in * 0.0254, H_in * 0.0254
DEFAULT_CAMERA_PARAMS = {
    'tag_size': 0.26416,  # meters (13/10*8 inches)
    'f': 2900 // 2,  # focal length in pixels, adjust till depth is about 11.03m
    'cx': 4032 // 2 // 2,  # camera center in pixels
    'cy': 3024 // 2 // 2,  # camera center in pixels
}  # second divide by 2 is because iOS app downsamples by 2
TAG_LOCATIONS_IN = {
    0: (18.75 + W_in, 34.25),  #
    2: (-22.25, 76.5),
    3: (-26.75, 34.0)
}
TAG_LOCATIONS = {k: (v[0] * 0.0254, v[1] * 0.0254) for k, v in TAG_LOCATIONS_IN.items()}
EE_LOCATION_IN = (18.5, 2)
EE_LOCATION = (EE_LOCATION_IN[0] * 0.0254, EE_LOCATION_IN[1] * 0.0254)

## Data Collection

In [ ]:
stop_signal = threading.Event()

def collect_data_cdpr():
    """Collects log data from the cable robot.  Returns tuple of Nx4 arrays for lengths and vels"""
    with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,1', port=PORT) as robot:
        robot.update()
        robot.all_data = []  # Flush
        while not stop_signal.is_set():
            robot.update()
            time.sleep(0.001)
        robot.send('d10,100')
        ts = np.array([d['timestamp'] for d in robot.all_data])
        motor_ls = np.array([[m.length for m in datum['motors']] for datum in robot.all_data])
        motor_ldots = np.array([[m.lengthdot for m in datum['motors']] for datum in robot.all_data])
        xy = np.array([[d['controller'].cur_x, d['controller'].cur_y] for d in robot.all_data])
        # Discard the initial data because the timestamps are probably wrong
        ok = (ts - ts[0]) > 0.03
        return ts[ok], motor_ls[ok], motor_ldots[ok], xy[ok]
def collect_data_april():
    all_detections = []
    with Detector(DEFAULT_CAMERA_PARAMS) as detector:
        while not stop_signal.is_set():
            (t_cap, t), detections = detector.detect()
            t_now = time.time()
            all_detections.append(((t_now - t + t_cap), detections))
            time.sleep(0.0001)  # relinquish execution to other threads
    # Extract tag locations
    anchor_locs = {k: [] for k in TAG_LOCATIONS}
    for _, dets in all_detections:
        for d in dets:
            if d.tag_id in TAG_LOCATIONS:
                R, t = d.Rt()
                anchor_locs[d.tag_id].append(t)
    anchor_locs = {k: np.hstack(v).T for k, v in anchor_locs.items()}
    # EE locations
    ee_locs = {}
    for timestamp, dets in all_detections:
        for d in dets:
            if d.tag_id == 4:
                R, t = d.Rt()
                ee_locs[timestamp] = t
    return all_detections, anchor_locs, ee_locs

def collect_data(duration=5):
    stop_signal.clear()
    with ThreadPoolExecutor() as executor:
        try:
            future1 = executor.submit(collect_data_cdpr)
            future2 = executor.submit(collect_data_april)
            concurrent.futures.wait([future1, future2], timeout=duration)
            stop_signal.set()
            t0 = time.time()
            while (t0 + 0.5 > time.time()) and not (future1.done() and future2.done()):
                time.sleep(0.1)
            if future1.done() and future2.done():
                return future1.result(), future2.result()
        except KeyboardInterrupt:
            stop_signal.set()
            t0 = time.time()
            while (t0 + 0.5 > time.time()) and not (future1.done() and future2.done()):
                time.sleep(0.1)
            if future1.done() and future2.done():
                return future1.result(), future2.result()
        print(f'Warning: Data collection might have hanged. cdpr done? {future1.done()}, april done? {future2.done()}')
        try:
            return future1.result(), future2.result()
        except KeyboardInterrupt:
            raise RuntimeError(f'Data collection failed: cdpr done? {future1.done()}, april done? {future2.done()}')

In [ ]:
# Test connection
(ts, ls, ldots, xys), (detections, anchor_locs, ee_locs) = collect_data(duration=0.5)
print(ts.shape, ls.shape, ldots.shape, xys.shape, len(detections))

mean_anchors = {k: np.mean(v, axis=0) for k, v in anchor_locs.items()}
print(mean_anchors)
print(TAG_LOCATIONS)
print('predicted width: ', mean_anchors[0][0] - mean_anchors[2][0])
print('actual width:    ', TAG_LOCATIONS[0][0] - TAG_LOCATIONS[2][0])

In [ ]:
if True:
    # Collect some data
    time.sleep(2)
    (ts, ls, ldots, xy),  (detections, anchor_locs, ee_locs) = collect_data(duration=65)
    print(f'Collected {len(ls)} samples')
    print(ls[-1])
    # Save/Load data (in case of ipynb failure)
    np.savez('/tmp/data.npz', ls=ls, ldots=ldots, est=xy)
    np.savez(f'/tmp/data_{time.time()}.npz', ls=ls, ldots=ldots, est=xy)
else:
    with np.load('/tmp/data.npz') as f:
        ls = f['ls']
        ldots = f['ldots']
        xy = f['est']

In [ ]:
ee_times = np.array(list(ee_locs.keys()))
ee_xyzs = np.hstack(list(ee_locs.values())).T
print(ee_times.shape, ee_xyzs.shape)

In [ ]:
# Plot data as a sanity check
fig, axes = plt.subplots(2, 3, sharex=False, figsize=(20, 8))
axes = axes.flatten()
axes[0].plot(ts - ts[0])
axes[1].plot(ls)
axes[2].plot(ldots);
axes[3].plot(*xy.T)
axes[3].axis('equal')
for v in [anchor_locs[0], anchor_locs[3], ee_xyzs]:
    axes[4].plot(v[:, 0], v[:, 1])
axes[4].axis('equal')
axes[0].set_title('times vs sample #')
axes[1].set_title('Cable Lengths (m)')
axes[2].set_title('Cable Velocities (m/s)')
axes[3].set_title('xy picture of trajectory')
fig.tight_layout()

## Calibration

In [ ]:
# First canonicalize the apriltag data
mean_anchors = {k: np.mean(v, axis=0) for k, v in anchor_locs.items()}
Rt = util.compute_Rt_from_anchors(TAG_LOCATIONS, mean_anchors)
print(Rt)
mean_anchors2 = util.apply(Rt, np.vstack(list(mean_anchors.values())))
print('predicted width: ', mean_anchors2[0][0] - mean_anchors2[2][0])
print('actual width:    ', TAG_LOCATIONS[0][0] - TAG_LOCATIONS[2][0])
ee_xys = util.apply(Rt, ee_xyzs) - [*EE_LOCATION, 0]

# Plot data as a sanity check
fig, axes = plt.subplots(1, 2, sharex=False, figsize=(8, 2))
axes[0].plot(*xy.T)
axes[0].axis('equal')
sc = axes[1].scatter(ee_xys[:, 0], ee_xys[:, 1], c=ee_xys[:, 2], s=1)
axes[1].axis('equal')
axes[0].set_title('Trajectory from proprioception')
axes[1].set_title('Trajectory from apriltags')
plt.colorbar(sc);

In [ ]:
# Interpolate the cable lengths to match the apriltag data
ls2 = util.align_timestamps(ts, ls, ee_times)
print(ls.shape, ls2.shape)
# plt.plot(ts, ls, linewidth=3)
# plt.plot(ee_times, ls2, 'k.', markersize=1);

In [ ]:
# Cable Length Calibration
W_EE, H_EE = 20.5 * 0.0254, 15.75 * 0.0254
INIT_MOUNTPOINTS = np.array([[W - W_EE, 0], [W - W_EE, H - H_EE], [0, H - H_EE], [0,
                                                                                  0]]).T.flatten()

# Do the calibration
solver = util.CableLengthCalibrationWithGt(ls2, ee_xys[:, :2], INIT_MOUNTPOINTS)
result = solver.calibrate()
print(result.success, result.message)
print(result.x)

In [ ]:
# Extract parameters and plot
mountPoints = solver.MOUNT_POINTS.reshape(2, 4)
lparams = solver.lparams
xs = solver.compute_x_estimates(subsample=10)
print('mount points:')
print(mountPoints)
print('lparams:')
print(lparams)

import matplotlib as mpl
mpl.rcParams['axes.titlesize'] = 22
mpl.rcParams['axes.labelsize'] = 18

plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(ls, ':')
plt.plot(solver.l_corr(lparams, ls), '-')
plt.title('Cable Lengths')
plt.legend([f'cable {i} uncalibrated' for i in range(4)] + [f'cable {i} calibrated' for i in range(4)])
plt.xlabel('Data Sample \# (roughly 2ms / sample)')
plt.ylabel('Cable Length (m)')
plt.subplot(122)
plt.plot(xs[:, 0], xs[:, 1], label='Estimated Trajectory')
plt.plot(ee_xys[:, 0], ee_xys[:, 1], label='Apriltag Trajectory')
plt.title('Estimated Trajectory')
plt.legend()
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.axis('equal');

In [ ]:
mountPoints
print('c54', *mountPoints.T.flatten().tolist(), sep=',')

In [ ]:
# Output in a format that can be sent directly to the cable robot
CARRIAGE_WIDTH, CARRIAGE_HEIGHT = 0.22377, 0.22377
mountPoints2 = mountPoints * 1
mountPoints2[0, 0:2] += CARRIAGE_WIDTH
mountPoints2[1, 1:3] += CARRIAGE_HEIGHT
print(mountPoints2) # sanity check
# lparams2 = np.vstack((lparams[0], np.ones(4), lparams[1]))
lparams2 = lparams
print(lparams2) # sanity check
print('-'*40)
print('c54', *mountPoints2.T.flatten().tolist(), sep=',')
print('c44', *lparams2.T.flatten().tolist(), sep=',')
print('-'*40)
print('c54', *mountPoints2.T.flatten().tolist(), sep=',', end=';')
print('c44', *lparams2.T.flatten().tolist(), sep=',')

In [ ]:
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port=PORT) as robot:
    robot.update()
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port=PORT, silent=False) as robot:
    robot.send('g6')
    s1 = 'c54,' + ','.join(map(str, mountPoints2.T.flatten()))
    s2 = 'c44,' + ','.join(map(str, lparams2.T.flatten()))
    robot.send(s1)
    robot.send(s2)
    for _ in range(50):
        robot.update()
        time.sleep(0.005)